**Advanced Lane Finding Project**

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

[//]: # (Image References)

[image1]: ./camera_cal_output/camera_cal/calibration7.jpg "Undistorted"
[image2]: ./test_images/test1.jpg "Road Transformed"
[image3]: ./examples/binary_combo_example.jpg "Binary Example"
[image4]: ./examples/warped_straight_lines.jpg "Warp Example"
[image5]: ./examples/color_fit_lines.jpg "Fit Visual"
[image6]: ./examples/example_output.jpg "Output"
[video1]: ./project_video.mp4 "Video"




### Camera Calibration

#### Briefly state how you computed the camera matrix and distortion coefficients. Provide an example of a distortion corrected calibration image.

To compute the camera matrix, I prepared a matrix of "object points" (real-world coordinates of the chessboard corners) and "image points" (2D coordinates in image space). I used OpenCV's ```findChessboardCorners()``` to populate the points, ```calibrateCamera()``` to calculate the transform (camera matrix) that maps the object points to the image points and the distortion coefficients, and ```undistort()``` to apply those coefficients to the image. The code can be found in cells 3, 4, & 5 in the Pipeline Prototype notebook.

<img src="./camera_cal_output/camera_cal/calibration5.jpg">

## PIPELINE (TEST IMAGES)

Source code can be found in the "Pipeline Prototype (Advanced Lane Line Finder)" notebook.

#### Provide an example of a distortion-corrected image.

##### Distorted
<img src="./test_images/test1.jpg">

##### Undistorted
<img src="./test_images_output/test_images/test1.jpg">

### Describe how (and identify where in your code) you used color transforms, gradients or other methods to create a thresholded binary image. Provide an example of a binary image result.

I used three layers to create my binary image. The code can be found in cell 23 of the Pipeline Prototype notebook.

The first layer (red channel) is a thresholded saturation channel in the HSV color space. I thresholded it between 75 and 255. This was able to identify the yellow lanes on dark roads.

The second layer (green channel) is a Sobel gradient with a 9-pixel kernel, which I took the magnitude of and scaled to 255. I then thresholded it between 60 and 255. This was able to identify the white lanes on dark roads.

The third layer (blue channel) is a thresholded value channel in the HSV color space. I thresholded it between 200 and 255. This, when combined with the saturation channel, was able to identify the yellow lanes on light roads.

I then created a simple mask.

When stacked together as seen below, I looked for green or purple to identify lane lines.

```# isolate pink or lime regions in RGB colorspace --> (r & b) | (g & b)```

```final_binary[((s_binary > 0) & (v_binary > 0)) | ((sobel_binary > 0) & (v_binary > 0))] = 1```


#### Filter output
<img src="./test_images_output/stacked_binary.jpg">

#### Binary image (mask borders shown intentionally)
<img src="./test_images_output/final_binary.jpg">

#### Describe how (and identify where in your code) you performed a perspective transform and provide an example of a transformed image.

The perspective transform can be found in cell 24 of the Pipeline Prototype notebook.

To pick the corners of the quadrilateral for my perspective transform, I used several images of straight roads and attempted to pick coordinates that would make all the images look as straight as possible.

<img src="test_images_output/binary_warped.jpg">

#### Describe how (and identify where in your code) you identified lane-line pixels and fit their positions with a polynomial?

The pixel detection function is in cell 107 and largely uses the boilerplate code from the lesson.

The polynomial fitting function is in cell 108, including one for determining a fit in real-world units, not just in pixels. It also uses mostly boilerplate code from the lesson. The polynomial is fit to ```x = A*y^2 + b*y + c```.

The actual calling of the functions happens in cell 112.

<img src="test_images_output/fitted_polynomial.jpg">

#### Describe how (and identify where in your code) you calculated the radius of curvature of the lane and the position of the vehicle with respect to center.

The function is in cell 110 and the function call is in cell 112.

The formula used is the formula provided in the lesson:

```((1 + ((2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2))**(3/2)) / np.abs(2*left_fit_cr[0])```

Note ```y``` is scaled by ```ym_per_pix``` to provide a result in meters.

#### Provide an example image of your result plotted back down onto the road such that the lane area is identified clearly.

<img src="test_images_output/final_output.jpg">

## PIPELINE (VIDEO)

Source code can be found in cell 25 of the "Advanced Lane Line Finder" notebook.

#### Provide a link to your final video output. Your pipeline should perform reasonably well on the entire project video (wobbly lines are ok but no catastrophic failures that would cause the car to drive off the road!)

In [10]:
from IPython.display import HTML

# project_video (final output)
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format("test_videos_output/project_video.mp4"))

Additionally, I used a smoothing filter (cell 11 of "Advanced Lane Line Finder" notebook) that was a simple ring buffer implementation of depth 40.

## DISCUSSION

#### Briefly discuss any problems / issues you faced in your implementation of this project.

The coordinates I chose for masking the image and for transforming the perspective are a little too aggressive. At multiple points in the video, one of the sides (usually the right) curves in from the lane toward the top. This is because in the transformed space, the right lane line has curved out the side of the frame.

#### Where will your pipeline likely fail?

It has shown to fail on tighter curves. Additionally, there are some areas of contrasting shadow (usually under trees) that, when combined with the color changes of the asphalt, seem to confuse my color space / gradient processing.

#### What could you do to make it more robust?

The transformation issue may be fixable with fine-tuning of the mask & transformation quadrilateral, but widening the upper side of the image mask allows for the detection of more false positive pixels, and so the overall accuracy even on straightaways drops. One possibility is that I could create an algorithm that changes parameters based on the scenario encountered (light tarmac / dark tarmac / lots of shadows / curved / straight / etc). This primarily turns into a classification problem, which I haven't learned about yet! (machine learning?)

One way to get around the classification problem would be to try several different parameter sets and come up with a certainty measurement based on number of detected pixels & their variance from an expected lane line (probably taken from the previous frame). I could then pick the best result based on minimizing uncertainty. This sounds a lot like a Kalman filter, which I look forward to learning about in a future lesson.

See around 0:30 in the video below for an example of the transformation / masking issue.

In [11]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format("test_videos_output/project_video.mp4_binary_warped.mp4"))